In [149]:
import pandas as pd

In [150]:
sales = pd.read_csv('sales.csv')
prices = pd.read_csv('prices.csv')

In [151]:
sales.head()

,product_id,ordered_at,quantity_ordered
0,3998909,9/18/18 17:51,1
1,3998909,9/18/18 12:52,1
2,3998909,9/18/18 11:33,1
3,3998909,9/18/18 18:47,1
4,3998909,9/18/18 17:36,1


In [152]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   product_id        175 non-null    int64 
 1   ordered_at        175 non-null    object
 2   quantity_ordered  175 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 4.2+ KB


In [153]:
prices.head()

,product_id,old_price,new_price,updated_at
0,64,270000,239000,9/10/18 16:37
1,3954203,60000,64000,9/11/18 11:54
2,3954203,60500,57500,9/17/18 22:59
3,3954203,64000,60500,9/15/18 3:49
4,3954203,68800,60000,9/10/18 16:32


In [154]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   product_id  14 non-null     int64 
 1   old_price   14 non-null     int64 
 2   new_price   14 non-null     int64 
 3   updated_at  14 non-null     object
dtypes: int64(3), object(1)
memory usage: 576.0+ bytes


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

In [155]:
#CONVERT TO DATETIME
sales['ordered_at'] = pd.to_datetime(sales['ordered_at'])
prices['updated_at'] = pd.to_datetime(prices['updated_at'])

In [156]:
#PRICES SORT (OLD_PRICE_SORT)
old_price_sort = prices[['product_id','old_price','updated_at']].rename(
    columns={'old_price':'price', 'updated_at':'date'}).sort_values('date')

old_price_sort.head()

,product_id,price,date
4,3954203,68800,2018-09-10 16:32:00
7,3998909,19000,2018-09-10 16:35:00
0,64,270000,2018-09-10 16:37:00
11,4085861,60000,2018-09-11 08:51:00
1,3954203,60000,2018-09-11 11:54:00


In [157]:
#PRICES SORT (NEW_PRICE_SORT)
new_price_sort = prices[['product_id','new_price','updated_at']].rename(
    columns={'new_price':'price', 'updated_at':'date'}).sort_values('date')

new_price_sort.head()

,product_id,price,date
4,3954203,60000,2018-09-10 16:32:00
7,3998909,17000,2018-09-10 16:35:00
0,64,239000,2018-09-10 16:37:00
11,4085861,53500,2018-09-11 08:51:00
1,3954203,64000,2018-09-11 11:54:00


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

In [158]:
#SALES SORT (SALES_SORT)
sales_sort = sales.rename(columns={'ordered_at':'date'}).sort_values('date')
sales_sort.head()

,product_id,date,quantity_ordered
86,3954203,2018-09-11 01:43:00,1
28,4085861,2018-09-11 06:26:00,1
26,4085861,2018-09-11 06:53:00,1
27,4085861,2018-09-11 08:24:00,1
123,4085861,2018-09-11 09:30:00,1


In [159]:
#FIND PRODUCT SALE WITH NEW PRICE
sales_price = pd.merge_asof(sales_sort, new_price_sort, by='product_id', on='date', direction='backward')
sales_price.head()

,product_id,date,quantity_ordered,price
0,3954203,2018-09-11 01:43:00,1,60000.0
1,4085861,2018-09-11 06:26:00,1,NaN
2,4085861,2018-09-11 06:53:00,1,NaN
3,4085861,2018-09-11 08:24:00,1,NaN
4,4085861,2018-09-11 09:30:00,1,53500.0


In [160]:
sales_new_price = sales_price.dropna()
sales_new_price.head()

,product_id,date,quantity_ordered,price
0,3954203,2018-09-11 01:43:00,1,60000.0
4,4085861,2018-09-11 09:30:00,1,53500.0
5,4085861,2018-09-11 11:06:00,1,53500.0
6,3954203,2018-09-11 11:11:00,1,60000.0
7,3954203,2018-09-11 11:11:00,1,60000.0


In [161]:
##FIND PRODUCT SALE WITH OLD PRICE
sales_null_price = sales_price.query('price!=price')
sales_null_price 

,product_id,date,quantity_ordered,price
1,4085861,2018-09-11 06:26:00,1,NaN
2,4085861,2018-09-11 06:53:00,1,NaN
3,4085861,2018-09-11 08:24:00,1,NaN


In [162]:
sales_drop_price = sales_null_price.drop(columns='price')
sales_drop_price

,product_id,date,quantity_ordered
1,4085861,2018-09-11 06:26:00,1
2,4085861,2018-09-11 06:53:00,1
3,4085861,2018-09-11 08:24:00,1


In [163]:
sales_old_price = pd.merge_asof(sales_drop_price, old_price_sort, by='product_id', on='date', direction='forward')
sales_old_price

,product_id,date,quantity_ordered,price
0,4085861,2018-09-11 06:26:00,1,60000
1,4085861,2018-09-11 06:53:00,1,60000
2,4085861,2018-09-11 08:24:00,1,60000


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

In [164]:
#FIND THE FINAL SALES PRICE
final_sales_price = pd.concat([sales_new_price,sales_old_price], ignore_index=True)
final_sales_price.head()

,product_id,date,quantity_ordered,price
0,3954203,2018-09-11 01:43:00,1,60000.0
1,4085861,2018-09-11 09:30:00,1,53500.0
2,4085861,2018-09-11 11:06:00,1,53500.0
3,3954203,2018-09-11 11:11:00,1,60000.0
4,3954203,2018-09-11 11:11:00,1,60000.0


In [165]:
#CALCULATE REVENUE AT EACH PRODUCT
final_sales_price['revenue'] = final_sales_price['quantity_ordered'] * final_sales_price['price']
final_sales_price

,product_id,date,quantity_ordered,price,revenue
0,3954203,2018-09-11 01:43:00,1,60000.0,60000.0
1,4085861,2018-09-11 09:30:00,1,53500.0,53500.0
2,4085861,2018-09-11 11:06:00,1,53500.0,53500.0
3,3954203,2018-09-11 11:11:00,1,60000.0,60000.0
4,3954203,2018-09-11 11:11:00,1,60000.0,60000.0
...,...,...,...,...,...
170,3954203,2018-09-18 21:26:00,1,57500.0,57500.0
171,3998909,2018-09-18 22:11:00,1,16500.0,16500.0
172,4085861,2018-09-11 06:26:00,1,60000.0,60000.0
173,4085861,2018-09-11 06:53:00,1,60000.0,60000.0


In [166]:
#CALCULATE TOTAL REVENUE AT EACH PRODUCT AT EACH PRICE
total_revenue = final_sales_price.groupby('product_id')[['revenue']].sum().reset_index()
total_revenue

,product_id,revenue
0,64,956000.0
1,3954203,877500.0
2,3998909,280500.0
3,4085861,8247500.0
